In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
import os

In [2]:
# ① 데이터 읽어오기
train_file = os.path.join('./models/intent', "Intent_train_data.csv")
df = pd.read_csv(train_file, delimiter=',', header=None)
df = df[[0, 1, 2]]

In [3]:
df.columns = ['query', 'intent', 'intent_info']
seq_len = list(map(lambda x : len(x.split(' ')), df['query']))
max(seq_len)  # 문장 최대길이

10

In [4]:
# 단어 시퀀스 벡터 크기
MAX_SEQ_LEN = max(seq_len)

def GlobalParams():
    global MAX_SEQ_LEN

In [5]:
# '질문(query)' 과 '의도(intent)'
queries = df['query'].tolist()
intents = df['intent'].tolist()

In [6]:
queries[:20]

['사이드메뉴',
 '사이드 메뉴',
 '빵 종류',
 '종류',
 '메뉴',
 '메뉴판',
 '사이드메뉴는 뭐가 있나요?',
 '사이드 메뉴는 뭐가 있나요?',
 '빵 종류는 뭐가 있나요?',
 '종류는 뭐가 있나요?',
 '메뉴는 뭐가 있나요?',
 '메뉴판는 뭐가 있나요?',
 '사이드메뉴는 어디 있나요?',
 '사이드 메뉴는 어디 있나요?',
 '빵 종류는 어디 있나요?',
 '종류는 어디 있나요?',
 '메뉴는 어디 있나요?',
 '메뉴판는 어디 있나요?',
 '사이드메뉴는 어디 있어요?',
 '사이드 메뉴는 어디 있어요?']

In [7]:
intents[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
# 전처리 모듈 불러오기
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'my_dict.tsv'))

In [9]:
# 단어 시퀀스 생성 (가장 시간 많이 걸림)
# 해당 단어에 매칭되는 번호로 시퀀스 생성

# ★ 시간 제법 걸림 * 몇십초 정도..

sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag=True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [10]:
sequences

[[7837, 67],
 [7837, 67],
 [64, 8941],
 [8941],
 [67],
 [67, 3344],
 [7837, 67, 552, 25],
 [7837, 67, 552, 25],
 [64, 8941, 552, 25],
 [8941, 552, 25],
 [67, 552, 25],
 [67, 3344, 552, 25],
 [7837, 67, 167, 25],
 [7837, 67, 167, 25],
 [64, 8941, 167, 25],
 [8941, 167, 25],
 [67, 167, 25],
 [67, 3344, 167, 25],
 [7837, 67, 167, 25],
 [7837, 67, 167, 25],
 [64, 8941, 167, 25],
 [8941, 167, 25],
 [67, 167, 25],
 [67, 3344, 167, 25],
 [7837, 67, 176],
 [7837, 67, 176],
 [64, 8941, 176],
 [8941, 176],
 [67, 176],
 [67, 3344, 176],
 [7837, 67, 176],
 [7837, 67, 176],
 [64, 8941, 176],
 [8941, 176],
 [67, 176],
 [67, 3344, 176],
 [7837, 67, 176],
 [7837, 67, 176],
 [64, 8941, 176],
 [8941, 176],
 [67, 176],
 [67, 3344, 176],
 [7837, 67, 176],
 [7837, 67, 176],
 [64, 8941, 176],
 [8941, 176],
 [67, 176],
 [67, 3344, 176],
 [7837, 67, 1851],
 [7837, 67, 1851],
 [64, 8941, 1851],
 [8941, 1851],
 [67, 1851],
 [67, 3344, 1851],
 [7837, 67, 1, 147, 1851, 19],
 [7837, 67, 1, 147, 1851, 19],
 [64, 89

In [11]:
# ② 단어 인덱스 시퀀스 벡터 
# 단어 시퀀스 벡터 크기 (MAX_SEQ_LEN 로 동일하게 맞추기, 패딩처리)
from config.GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [12]:
padded_seqs

array([[ 7837,    67,     0, ...,     0,     0,     0],
       [ 7837,    67,     0, ...,     0,     0,     0],
       [   64,  8941,     0, ...,     0,     0,     0],
       ...,
       [10258,   188,     0, ...,     0,     0,     0],
       [ 3599,  3566,     0, ...,     0,     0,     0],
       [    1,     0,     0, ...,     0,     0,     0]])

In [13]:
len(p.word_index)

17751

In [43]:
# ③ 학습용, 검증용, 테스트용 데이터셋 생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents)) # 패딩처리된 시퀀스와 의도(intent) 리스트 전체를 데이터셋 객체로
ds = ds.shuffle(len(queries)) # 랜덤 섞기

# 학습셋:검셋:테스트셋 = 7:2:1
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 10
VOCAB_SIZE = len(p.word_index) + 1 #전체 단어 개수

## train_ds

In [44]:
# ④ CNN 모델 정의
# keras 함수형 모델 방식으로 구현
input_layer = Input(shape=(MAX_SEQ_LEN,))  # 입력크기
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(12, name='logits')(dropout_hidden)  # 최종적으로 12가의 의도 클래스를 분류. 결과로 나온 값(logits) 을을 점수(score) 라 부른다
predictions = Dense(12, activation=tf.nn.softmax)(logits)


In [45]:
# ⑤ 모델 생성 
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 10, 128)      2272256     ['input_2[0][0]']                
                                                                                                  
 dropout_2 (Dropout)            (None, 10, 128)      0           ['embedding_1[0][0]']            
                                                                                                  
 conv1d_3 (Conv1D)              (None, 8, 128)       49280       ['dropout_2[0][0]']              
                                                                                            

In [47]:
# 모델학습
# ★ 시간 걸림 ★
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/10
1248/1248 [==============================] - 40s 31ms/step - loss: 0.3164 - accuracy: 0.8998 - val_loss: 0.0331 - val_accuracy: 0.9875
Epoch 2/10
1248/1248 [==============================] - 41s 33ms/step - loss: 0.0575 - accuracy: 0.9817 - val_loss: 0.0252 - val_accuracy: 0.9913
Epoch 3/10
1248/1248 [==============================] - 42s 33ms/step - loss: 0.0422 - accuracy: 0.9861 - val_loss: 0.0268 - val_accuracy: 0.9895
Epoch 4/10
1248/1248 [==============================] - 40s 32ms/step - loss: 0.0413 - accuracy: 0.9864 - val_loss: 0.0245 - val_accuracy: 0.9898
Epoch 5/10
1248/1248 [==============================] - 39s 31ms/step - loss: 0.0375 - accuracy: 0.9874 - val_loss: 0.0167 - val_accuracy: 0.9934
Epoch 6/10
1248/1248 [==============================] - 39s 32ms/step - loss: 0.0336 - accuracy: 0.9879 - val_loss: 0.0198 - val_accuracy: 0.9903
Epoch 7/10
1248/1248 [==============================] - 39s 31ms/step - loss: 0.0318 - accuracy: 0.9886 - val_loss: 0.0174 -

In [48]:
# ⑦ 모델 평가(테스트 데이터 셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

179/179 [==============================] - 0s 2ms/step - loss: 0.0250 - accuracy: 0.9871
Accuracy: 98.708951
loss: 0.024974


In [49]:
# ⑧ 모델 저장
model.save(os.path.join('./models/intent','intent_model.h5'))

## 의도분류 모듈 테스트

In [50]:
from utils.Preprocess import Preprocess
from models.intent.IntentModel import IntentModel
import os

p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

In [51]:
# ⑧ 모델 저장
model.save(os.path.join('./models/intent','intent_model.h5'))

In [52]:
intent = IntentModel(model_name=os.path.join('./models/intent', 'intent_model.h5'), preprocess=p)

In [67]:
query = "초코라떼"

In [68]:
predict = intent.predict_class(query)
predict_label = intent.labels[predict]

In [69]:
print(query)
print('의도 예측 클래스 : ', predict)
print('의도 예측 레이블 : ', predict_label)

초코라떼
의도 예측 클래스 :  10
의도 예측 레이블 :  주문취소
